<a href="https://colab.research.google.com/github/VaibhavDwivedi01/Galgotias_University_ML_Lab_Solution/blob/main/ML_Experiment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
boston = fetch_openml(name="boston", version=1, as_frame=True)
X = boston.data.astype(float)
y = boston.target.astype(float)
print("=== Top 10 rows of Boston Housing Dataset ===")
print(X.head(10))


=== Top 10 rows of Boston Housing Dataset ===
      CRIM    ZN  INDUS  CHAS    NOX     RM    AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575   65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421   78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185   61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998   45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147   54.2  6.0622  3.0  222.0   
5  0.02985   0.0   2.18   0.0  0.458  6.430   58.7  6.0622  3.0  222.0   
6  0.08829  12.5   7.87   0.0  0.524  6.012   66.6  5.5605  5.0  311.0   
7  0.14455  12.5   7.87   0.0  0.524  6.172   96.1  5.9505  5.0  311.0   
8  0.21124  12.5   7.87   0.0  0.524  5.631  100.0  6.0821  5.0  311.0   
9  0.17004  12.5   7.87   0.0  0.524  6.004   85.9  6.5921  5.0  311.0   

   PTRATIO       B  LSTAT  
0     15.3  396.90   4.98  
1     17.8  396.90   9.14  
2     17.8  392.83   4.03  
3     18.7  394.63   2.94  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
models = {"Linear Regression": LinearRegression(),
"Ridge (alpha=1)": Ridge(alpha=1, random_state=42),
"Lasso (alpha=0.1)": Lasso(alpha=0.1, random_state=42, max_iter=10000)}
results = []
for name, model in models.items():
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  results.append({
  "Model": name,
  "MSE": mse,
  "Coefficients": model.coef_
})
df_results = pd.DataFrame(results)
coef_df = pd.DataFrame({
"Feature": X.columns,
"Linear": results[0]["Coefficients"],
"Ridge": results[1]["Coefficients"],
"Lasso": results[2]["Coefficients"]
})
fig_mse_table = go.Figure(
data=[go.Table(
header=dict(values=list(df_results[["Model", "MSE"]].columns),
fill_color='lightblue',
align='center'),
cells=dict(values=[df_results["Model"], df_results["MSE"].round(3)],
fill_color='white',
align='center'))
]
)
fig_mse_table.update_layout(title="MSE Comparison Table")
fig_mse_table.show()


In [ ]:
fig_coef_table = go.Figure(
data=[go.Table(
header=dict(values=list(coef_df.columns),
fill_color='lightgreen',
align='center'),cells=dict(values=[coef_df[col].round(3) for col in coef_df.columns],
fill_color='white',
align='center'))])
fig_coef_table.update_layout(title="Regression Coefficients Comparison Table")
fig_coef_table.show()

In [ ]:
fig = go.Figure()

for col in ["Linear", "Ridge", "Lasso"]:
    fig.add_trace(
        go.Bar(
            x=coef_df["Feature"],
            y=coef_df[col],
            name=col
        )
    )

fig.update_layout(
    title="Comparison of Coefficients: Linear vs Ridge vs Lasso",
    xaxis_title="Features",
    yaxis_title="Coefficient Value",
    barmode="group",
    height=600,
    width=1000
)

fig.show()


In [ ]:
alphas = [0.01, 0.1, 1, 10, 100]
ridge_mses, lasso_mses = [], []

for a in alphas:
    ridge = Ridge(alpha=a, random_state=42)
    lasso = Lasso(alpha=a, random_state=42, max_iter=10000)

    ridge.fit(X_train, y_train)
    lasso.fit(X_train, y_train)

    ridge_mses.append(mean_squared_error(y_test, ridge.predict(X_test)))
    lasso_mses.append(mean_squared_error(y_test, lasso.predict(X_test)))

fig_alpha = go.Figure()

fig_alpha.add_trace(go.Scatter(
    x=alphas, y=ridge_mses, mode='lines+markers', name='Ridge'
))
fig_alpha.add_trace(go.Scatter(
    x=alphas, y=lasso_mses, mode='lines+markers', name='Lasso'
))

fig_alpha.update_layout(
    title="Regularization Effect: MSE vs Alpha",
    xaxis_title="Alpha (λ)",
    yaxis_title="Mean Squared Error",
    xaxis_type="log",
    height=500,
    width=800
)

fig_alpha.show()
